<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_TF_qasc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Jan  4 06:19:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd 
import json
import random
import math
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Then set a random seed for deterministic results/reproducability.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!gsutil cp -r \
  gs://unifiedqa/data/qasc/ \
  gs://unifiedqa/data/qasc_test/ \
  gs://unifiedqa/data/qasc_with_ir/ \
  gs://unifiedqa/data/qasc_with_ir_test/ \
  .

Copying gs://unifiedqa/data/qasc/counts.json...
Copying gs://unifiedqa/data/qasc/dev.tsv...
Copying gs://unifiedqa/data/qasc/dev_meta.tsv...
Copying gs://unifiedqa/data/qasc/test.tsv...
- [4 files][352.9 KiB/352.9 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://unifiedqa/data/qasc/test_meta.tsv...
Copying gs://unifiedqa/data/qasc/train.tsv...
Copying gs://unifiedqa/data/qasc/train_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/counts.json...
Copying gs://unifiedqa/data/qasc_test/dev.tsv...
Copying gs://unifiedqa/data/qasc_test/dev_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/test.tsv...
Copying gs://unifiedqa/data/qasc_test/test_meta.tsv...
Copying gs://unifiedqa/data/qasc_test/train.tsv...
Copying gs://unifiedqa

In [ ]:
def read_file(file, split):
    fout = open(f"./{split}.tsv", "w")
    fout_meta = open(f"./{split}_meta.tsv", "w")
    with open(file) as f:
        for line in f.readlines():
            json_line = json.loads(line)

            candidates_str = " ".join([f"({x['label']}) {x['text']}" for x in json_line['question']['choices']])
            if split != "test":
                selected_ans_string = [x['text'] for x in json_line['question']['choices'] if
                                        json_line['answerKey'] == x['label']]
                assert len(selected_ans_string) == 1, f"{len(selected_ans_string)} -- {json_line['answerKey']}"
            json_line['question']['stem'] = json_line['question']['stem'].replace("\t", " ").replace("\n", "")
            candidates_str = candidates_str.replace("\t", " ").replace("\n", "")

            if split == "test":
                fout_meta.write(f"{json_line['id']}\t-\n")
                fout.write(f"{json_line['question']['stem']} \\n {candidates_str}\t-\n")
            else:
                fout_meta.write(f"{json_line['id']}\t{json_line['answerKey']}\n")
                selected_ans_string[0] = selected_ans_string[0].replace("\t", " ").replace("\n", "")
                fout.write(f"{json_line['question']['stem']} \\n {candidates_str}\t{selected_ans_string[0]}\n")

In [ ]:
#dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None)
dataset = pd.read_csv("/content/qasc/train.tsv", sep='\t', header=None)
dataset.columns = ["sentence", "label"]

In [ ]:
dataset

,sentence,label
0,What type of water formation is formed by clou...,beads
1,Where do beads of water come from? \n (A) Too ...,Vapor turning into a liquid
2,What forms beads of water? \n (A) Necklaces. ...,Steam.
3,what kind of beads are formed from vapor conde...,h2o
4,what kind of beads are formed by their vapor c...,h2o
...,...,...
8129,Chitin can be used for protection by whom? \n ...,Fish
8130,Which type of animal uses plates for protectio...,reptiles
8131,What are used for protection by fish? \n (A) s...,scales
8132,What are pangolins covered in? \n (A) tunicate...,protection


In [ ]:
!gsutil -m cp -r gs://unifiedqa/data/squad2/dev.tsv .

Copying gs://unifiedqa/data/squad2/dev.tsv...
/ [1/1 files][ 10.3 MiB/ 10.3 MiB] 100% Done                                    
Operation completed over 1 objects/10.3 MiB.                                     


In [ ]:
!gsutil -m cp -r \
  gs://unifiedqa/data/ambigqa/ \
  .

Copying gs://unifiedqa/data/ambigqa/counts.json...
Copying gs://unifiedqa/data/ambigqa/dev.tsv...
Copying gs://unifiedqa/data/ambigqa/train_meta.tsv...
Copying gs://unifiedqa/data/ambigqa/train.tsv...
Copying gs://unifiedqa/data/ambigqa/dev_meta.tsv...
/ [5/5 files][  2.8 MiB/  2.8 MiB] 100% Done                                    
Operation completed over 5 objects/2.8 MiB.                                      


In [ ]:
read_file("/content/dev_rand_split.jsonl", "dev")

Instantiate our German and English spaCy models.

In [ ]:
!python3 -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9MB 14.2MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=db821aad79b4fff44bc3f6f93013824542eeaa2f6955efdd9e116ac8d6dc17bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-v0mzfpvk/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
import de_core_news_sm

In [ ]:
spacy_de =  de_core_news_sm.load()
spacy_en = spacy.load('en')

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [ ]:
dataset

,sentence,label
0,What type of water formation is formed by clou...,beads
1,Where do beads of water come from? \n (A) Too ...,Vapor turning into a liquid
2,What forms beads of water? \n (A) Necklaces. ...,Steam.
3,what kind of beads are formed from vapor conde...,h2o
4,what kind of beads are formed by their vapor c...,h2o
...,...,...
8129,Chitin can be used for protection by whom? \n ...,Fish
8130,Which type of animal uses plates for protectio...,reptiles
8131,What are used for protection by fish? \n (A) s...,scales
8132,What are pangolins covered in? \n (A) tunicate...,protection


In [ ]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

In [ ]:
#dataset = dataset_df
fields = [('sentence', Sentence),('label',Label)]
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  9462
Size of label vocab :  3610
Top 10 words appreared repeatedly : [(')', 55314), ('(', 55313), ('A', 7331), ('H', 6944), ('C', 6923), ('B', 6921), ('D', 6916), ('\\n', 6914), ('E', 6914), ('F', 6914)]
Labels :  defaultdict(<function _default_unk_index at 0x7f6c2fa93f28>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'of': 3, 'a': 4, 'and': 5, 'energy': 6, 'water': 7, '.': 8, 'to': 9, 'heat': 10, 'in': 11, 'plants': 12, 'it': 13, 'food': 14, 'animals': 15, 'cells': 16, 'light': 17, 'humans': 18, 'soil': 19, 'air': 20, 'body': 21, 'The': 22, 'A': 23, 'bacteria': 24, "'s": 25, 'Something': 26, 'chemical': 27, 'them': 28, ',': 29, 'an': 30, 'rain': 31, 'wind': 32, '-': 33, 'oxygen': 34, 'It': 35, 'insects': 36, 'on': 37, 'sunlight': 38, 'blood': 39, 'life': 40, 'their': 41, 'system': 42, 'organisms': 43, 'reproduction': 44, 'sun': 45, 'electricity': 46, 'is': 47, 'temperature': 48, 'acid': 49, 'viruses': 50, 'skin': 51, 'trees': 52, 'cancer': 53, 'from': 54, 'pressure': 55, 

In [ ]:
Sentence.vocab.stoi['new construction'], Label.vocab.stoi['new construction'] 

(0, 0)

Load our data.

We'll also print out an example just to double check they're not reversed.

Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

Finally, define the `device` and create our iterators.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE=128
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

In [ ]:
iter_one = next(iter(train_iterator))

In [ ]:
iter_one.label.shape

torch.Size([9, 128])

In [ ]:
for entry in (iter_one.sentence[0][0,:]):
    print(entry.item())

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        text, text_lengths = src
        embedded = self.dropout(self.embedding(text))
        # packed_seq = nn.utils.rnn.pack_padded_sequence(embedded, 
        #                                                text_lengths.cpu(),
        #                                                batch_first=False,
        #                                                enforce_sorted=False)        
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state! 
        ## outputs is a packed sequence but since it is not used we will not 
        ## unpack it         
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Model Definition

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        # print(embedded.shape, hidden.shape, context.shape)
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model


In [ ]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 256
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9462, 256)
    (rnn): GRU(256, 256)
    (dropout): Dropout(p=0.8, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3610, 256)
    (rnn): GRU(512, 256)
    (fc_out): Linear(in_features=768, out_features=3610, bias=True)
    (dropout): Dropout(p=0.8, inplace=False)
  )
)

We print out the number of parameters.


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,108,634 trainable parameters


We initiaize our optimizer.

In [ ]:
from torch.optim.lr_scheduler import StepLR, OneCycleLR, MultiStepLR, CyclicLR, ReduceLROnPlateau

optimizer = optim.Adam(model.parameters(), lr=0.0009893272, weight_decay=0.0001)
scheduler = OneCycleLR(optimizer, 
                        0.001, 
                        epochs=100, 
                        cycle_momentum=False, 
                        steps_per_epoch=len(train_iterator), 
                        #base_momentum=config.momentum,
                        #max_momentum=0.95, 
                        pct_start=0.208,
                        # anneal_strategy=config.anneal_strategy,
                        div_factor=100,
                        # final_div_factor=config.final_div_factor
                        )
# 0.000981989011942079

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [ ]:
#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.sentence
        trg = batch.label
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        # print("output_dim:", output.shape)
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        # print("output_dim before loss:", output.shape, trg.shape)
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        # scheduler.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 1e-06
    lr: 9.999999999999159e-07
    max_lr: 0.001
    min_lr: 9.999999999999999e-11
    weight_decay: 0.0001
)

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)

In [ ]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
model.load_state_dict(torch.load('tut2-model.pt'))
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | LR: {scheduler.get_last_lr()}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.495 | Train PPL: 661.664 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.083 |  Val. PPL: 1191.807
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.465 | Train PPL: 642.095 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.080 |  Val. PPL: 1188.536
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.453 | Train PPL: 634.840 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.078 |  Val. PPL: 1185.041
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.446 | Train PPL: 629.997 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.075 |  Val. PPL: 1181.794
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.440 | Train PPL: 626.309 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.072 |  Val. PPL: 1178.952
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.435 | Train PPL: 623.474 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.070 |  Val. PPL: 1176.561
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.432 | Train PPL: 621.194 | LR: [1.0000000000000026e-05]
	 Val. Loss: 7.069 |  Val. PPL: 1174.570
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.428 | Tra

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01, amsgrad=True)

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0.01
)

In [ ]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = 7.02376618385315
model.load_state_dict(torch.load('tut2-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.311 | Train PPL: 550.375
	 Val. Loss: 7.024 |  Val. PPL: 1122.716
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.310 | Train PPL: 549.776
	 Val. Loss: 7.023 |  Val. PPL: 1122.437
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.308 | Train PPL: 549.150
	 Val. Loss: 7.023 |  Val. PPL: 1122.156
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.307 | Train PPL: 548.532
	 Val. Loss: 7.023 |  Val. PPL: 1121.894
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.306 | Train PPL: 547.947
	 Val. Loss: 7.023 |  Val. PPL: 1121.638
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.305 | Train PPL: 547.307
	 Val. Loss: 7.022 |  Val. PPL: 1121.387
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.304 | Train PPL: 546.812
	 Val. Loss: 7.022 |  Val. PPL: 1121.151
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.303 | Train PPL: 546.084
	 Val. Loss: 7.022 |  Val. PPL: 1120.922
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.302 | Train PPL: 545.529
	 Val. Loss: 7.022 |  Val. PPL: 1120.705
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.301 | Train PPL:

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-8, weight_decay=0.05)
# scheduler = OneCycleLR(optimizer, 
#                         0.000001, 
#                         epochs=100, 
#                         cycle_momentum=False, 
#                         steps_per_epoch=len(train_iterator), 
#                         #base_momentum=config.momentum,
#                         #max_momentum=0.95, 
#                         pct_start=0.5,
#                         # anneal_strategy=config.anneal_strategy,
#                         div_factor=1000,
#                         # final_div_factor=config.final_div_factor
#                         )

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-09
    weight_decay: 5e-05
)

In [ ]:
print(best_valid_loss)

6.83048825263977


In [ ]:
N_EPOCHS = 50
CLIP = 1
print(best_valid_loss)
model.load_state_dict(torch.load('tut2-model.pt'))

best_valid_loss = 6.830108547210694 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

6.830108547210694
Epoch: 01 | Time: 0m 1s
	Train Loss: 6.400 | Train PPL: 602.021
	 Val. Loss: 6.830 |  Val. PPL: 925.291
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.400 | Train PPL: 602.059
	 Val. Loss: 6.830 |  Val. PPL: 925.291
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.400 | Train PPL: 602.069
	 Val. Loss: 6.830 |  Val. PPL: 925.291
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.401 | Train PPL: 602.150
	 Val. Loss: 6.830 |  Val. PPL: 925.290
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.401 | Train PPL: 602.179
	 Val. Loss: 6.830 |  Val. PPL: 925.290
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.401 | Train PPL: 602.210
	 Val. Loss: 6.830 |  Val. PPL: 925.289
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.401 | Train PPL: 602.225
	 Val. Loss: 6.830 |  Val. PPL: 925.289
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.401 | Train PPL: 602.268
	 Val. Loss: 6.830 |  Val. PPL: 925.288
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.401 | Train PPL: 602.376
	 Val. Loss: 6.830 |  Val. PPL: 925.288
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.401 | T

In [ ]:
print(best_valid_loss)

6.830108547210694


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.000001, weight_decay=0.1)
scheduler = OneCycleLR(optimizer, 
                        config.ocp_max_lr, 
                        epochs=config.epochs, 
                        cycle_momentum=cycle_momentum, 
                        steps_per_epoch=len(trainloader), 
                        base_momentum=config.momentum,
                        max_momentum=0.95, 
                        pct_start=0.208,
                        anneal_strategy=config.anneal_strategy,
                        div_factor=config.div_factor,
                        final_div_factor=config.final_div_factor
                        )

In [ ]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = 5.484 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.445 | Train PPL:  11.535
	 Val. Loss: 5.488 |  Val. PPL: 241.844
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.425 | Train PPL:  11.305
	 Val. Loss: 5.480 |  Val. PPL: 239.949
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.403 | Train PPL:  11.058
	 Val. Loss: 5.477 |  Val. PPL: 239.027
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.374 | Train PPL:  10.738
	 Val. Loss: 5.471 |  Val. PPL: 237.608
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.345 | Train PPL:  10.437
	 Val. Loss: 5.465 |  Val. PPL: 236.272
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.324 | Train PPL:  10.217
	 Val. Loss: 5.460 |  Val. PPL: 235.171
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.302 | Train PPL:   9.992
	 Val. Loss: 5.450 |  Val. PPL: 232.854
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.273 | Train PPL:   9.711
	 Val. Loss: 5.454 |  Val. PPL: 233.587
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.253 | Train PPL:   9.516
	 Val. Loss: 5.447 |  Val. PPL: 231.988
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.221 | Train PPL:   9.218


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-6, weight_decay=0.0005)

In [ ]:
best_valid_loss

4.96515295902888

In [ ]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut2-model.pt'))
best_valid_loss = 4.9634 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.041 | Train PPL:   7.702
	 Val. Loss: 4.956 |  Val. PPL: 141.985
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.037 | Train PPL:   7.665
	 Val. Loss: 4.958 |  Val. PPL: 142.298
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.030 | Train PPL:   7.611
	 Val. Loss: 4.958 |  Val. PPL: 142.280
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.035 | Train PPL:   7.649
	 Val. Loss: 4.959 |  Val. PPL: 142.447
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.033 | Train PPL:   7.634
	 Val. Loss: 4.960 |  Val. PPL: 142.554
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.027 | Train PPL:   7.589
	 Val. Loss: 4.960 |  Val. PPL: 142.524
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.032 | Train PPL:   7.628
	 Val. Loss: 4.960 |  Val. PPL: 142.545
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.031 | Train PPL:   7.622
	 Val. Loss: 4.959 |  Val. PPL: 142.499
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.027 | Train PPL:   7.590
	 Val. Loss: 4.960 |  Val. PPL: 142.523
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.027 | Train PPL:   7.595


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-8, weight_decay=0.005)

In [ ]:
N_EPOCHS = 20
CLIP = 1
model.load_state_dict(torch.load('tut2-model.pt'))
best_valid_loss = 4.890922844409943 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.241 | Train PPL:   9.402
	 Val. Loss: 4.890 |  Val. PPL: 132.906
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.248 | Train PPL:   9.473
	 Val. Loss: 4.890 |  Val. PPL: 132.938
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.255 | Train PPL:   9.537
	 Val. Loss: 4.890 |  Val. PPL: 132.961
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.249 | Train PPL:   9.477
	 Val. Loss: 4.891 |  Val. PPL: 133.118
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.259 | Train PPL:   9.572
	 Val. Loss: 4.891 |  Val. PPL: 133.125
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.255 | Train PPL:   9.532
	 Val. Loss: 4.891 |  Val. PPL: 133.151
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.262 | Train PPL:   9.598
	 Val. Loss: 4.892 |  Val. PPL: 133.192
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.263 | Train PPL:   9.611
	 Val. Loss: 4.892 |  Val. PPL: 133.223
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.267 | Train PPL:   9.647
	 Val. Loss: 4.892 |  Val. PPL: 133.251
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.265 | Train PPL:   9.628


In [ ]:
N_EPOCHS = 20
CLIP = 1
model.load_state_dict(torch.load('/content/CommonQA.pt'))
best_valid_loss = 4.890922844409943 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 2s
	Train Loss: 2.241 | Train PPL:   9.402
	 Val. Loss: 4.890 |  Val. PPL: 132.904
Epoch: 02 | Time: 0m 2s
	Train Loss: 2.242 | Train PPL:   9.415
	 Val. Loss: 4.890 |  Val. PPL: 132.907
Epoch: 03 | Time: 0m 2s
	Train Loss: 2.249 | Train PPL:   9.474
	 Val. Loss: 4.890 |  Val. PPL: 132.911
Epoch: 04 | Time: 0m 2s
	Train Loss: 2.249 | Train PPL:   9.475
	 Val. Loss: 4.890 |  Val. PPL: 132.913
Epoch: 05 | Time: 0m 2s
	Train Loss: 2.248 | Train PPL:   9.470
	 Val. Loss: 4.890 |  Val. PPL: 132.915
Epoch: 06 | Time: 0m 2s
	Train Loss: 2.247 | Train PPL:   9.463
	 Val. Loss: 4.890 |  Val. PPL: 132.918
Epoch: 07 | Time: 0m 2s
	Train Loss: 2.245 | Train PPL:   9.439
	 Val. Loss: 4.890 |  Val. PPL: 132.920
Epoch: 08 | Time: 0m 2s
	Train Loss: 2.248 | Train PPL:   9.465
	 Val. Loss: 4.890 |  Val. PPL: 132.924
Epoch: 09 | Time: 0m 2s
	Train Loss: 2.242 | Train PPL:   9.415
	 Val. Loss: 4.890 |  Val. PPL: 132.927
Epoch: 10 | Time: 0m 2s
	Train Loss: 2.252 | Train PPL:   9.511


#### Final Perplexity = 
Epoch: 01 | Time: 0m 2s
	Train Loss: 2.241 | Train PPL:   9.402
	 Val. Loss: 4.890 |  Val. PPL: 132.904

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')